In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from time_templates.utilities.plot import plot_profile_1d
import uproot
import pandas as pd
from collections import defaultdict
from scipy.interpolate import interp1d
from scipy import integrate as scint
from re import findall

from read_Espec import load_spec

# plt.rcParams.update({"text.usetex": True, "font.size": 20, "font.family":"sans"})
# plt.style.use('science')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
markers = ['s', 'o', '^', 'v', 'x', '*']
lss = ['-', '--', ':', '-.']

ECH = 54e6
EVEM = 316e6

In [ ]:
def read_files(data_dir):
    dd = defaultdict(list)
    for basedir, _, files in os.walk(data_dir):
        for file in files:
            load_spec(os.path.join(basedir, file), dd, ECH, EVEM)

    df = pd.DataFrame(dd)   
#     df['?']
    return df

In [ ]:
data_dir = '/home/mart/auger/data/CorSims_aab/Espec_hists/new'

df_p = read_files(data_dir+'/EPOS/proton')
df_Fe = read_files(data_dir+'/EPOS/iron')

Plot random spec

In [ ]:

hist = uproot.open('/home/mart/auger/data/CorSims_aab/Espec_hists/new/EPOS/proton/EPOS-fluka_p_19_0_2_DAT000002_Espec_hists.root')
ir = 11

f, ax = plt.subplots(1, figsize=(8, 6))

colors = ['y', 'r', 'b', 'g', 'm', 'k']

dict_total = {}

for key, c in zip(hist.keys(), colors):
    spec, lgr_bins, psi_bins, lgE_pkin_bins = hist[key].to_numpy(flow=True)
    dict_total[key] = spec.sum()
    lgE_pkin_bins[0] = np.log10(0.05e6)
    lgE_pkin_bins[-1] = 13    
    lgE = (lgE_pkin_bins[1:] + lgE_pkin_bins[:-1])/2

    ax.plot(lgE, spec[ir].sum(axis=0), color=c, drawstyle='steps-mid')

ax.set_yscale('log')

from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='y'),
                Line2D([0], [0], color='r'),
                Line2D([0], [0], color='b'),
                Line2D([0], [0], color='g'),
                Line2D([0], [0], color='m'),
                Line2D([0], [0], color='k')]

ax.legend(custom_lines, ['photons', 'electrons', 'muons', 'protons', 'neutrons', 'other'], fontsize=18)

ax.set_xlabel('$\\log_{10}(E_{\\rm kin}/\\mathrm{eV})$')
ax.set_ylabel('$\\frac{dN}{d\\lg{\\left(E/\\mathrm{eV}\\right)}}$')

ax.set_title(f'${10**lgr_bins[ir]:.0f} < r/\\mathrm{{m}} < {10**lgr_bins[ir+1]:.0f}$', fontsize=18)
ax.set_ylim([1e2, None])
ax.set_xlim([6, 10.5])
plt.tight_layout()

# plt.savefig("Espec_EPOS_p_lgE19_Z0_r1000.pdf", bbox_inches='tight')

In [ ]:
f, (ax, ax2) = plt.subplots(1, 2, figsize=(15, 6))

colors = ['y', 'r', 'b', 'g']
labels = ['photons', 'electrons', 'muons']
for ir, ls in zip([5, 10, 15], ['-', '--', ':']):

    for key, c in zip(hist.keys(), colors):
        spec, lgr_bins, psi_bins, lgE_pkin_bins = hist[key].to_numpy(flow=True)

        lgE_pkin_bins[0] = np.log10(0.05e6)
        lgE_pkin_bins[-1] = 13    
        lgE = (lgE_pkin_bins[1:] + lgE_pkin_bins[:-1])/2
        if 'hadron' in key:
            continue
        spec = spec[ir].sum(axis=0)
        ax.plot(lgE, spec/spec.sum(), color=c, drawstyle='steps-mid', ls=ls)
    labels.append(f'$ {10**lgr_bins[ir]:.0f} < r/\\mathrm{{m}} < {10**lgr_bins[ir+1]:.0f}$')

ax.set_yscale('log')
ax2.set_yscale('log')
custom_lines = [Line2D([0], [0], color='y'),
                Line2D([0], [0], color='r'),
                Line2D([0], [0], color='b'),
                Line2D([0], [0], color='k', ls='-'),
                Line2D([0], [0], color='k', ls='--'),
                Line2D([0], [0], color='k', ls=':')]

ax.legend(custom_lines, labels, fontsize=16, ncol=2, loc=2)
ax.set_xlabel('$\\log_{10}(E_{\\rm kin}/\\mathrm{eV})$')
ax.set_ylabel('$\\frac{1}{N}\\frac{dN}{d\\lg{\\left(E/\\mathrm{eV}\\right)}}$')

ax.set_xlim([4.5, 10.5])
ax.set_ylim([1e-5, 2])

labels = []

for theta, ls in zip([0, 30, 45], ['-', '--', ':']):
    hist = uproot.open(f'/home/mart/auger/data/CorSims_aab/Espec_hists/proton/19/Z{theta}/EPOS-fluka_p_19_{theta}_2_DAT000002_Espec_hists.root')
    for key, c in zip(hist.keys(), colors):
        spec, lgr_bins, psi_bins, lgE_pkin_bins = hist[key].to_numpy(flow=True)

        lgE_pkin_bins[0] = np.log10(0.05e6)
        lgE_pkin_bins[-1] = 13    
        lgE = (lgE_pkin_bins[1:] + lgE_pkin_bins[:-1])/2
        if 'hadron' in key:
            continue
        spec = spec[10].sum(axis=0)
        ax2.plot(lgE, spec/spec.sum(), color=c, drawstyle='steps-mid', ls=ls)

    labels.append(f'$\\theta = {theta:.0f}^\\circ$')
    
custom_lines = [Line2D([0], [0], color='k', ls='-'),
                Line2D([0], [0], color='k', ls='--'),
                Line2D([0], [0], color='k', ls=':')]
ax2.legend(custom_lines, labels, fontsize=16, loc=2)
ax2.set_xlabel('$\\log_{10}(E_{\\rm kin}/\\mathrm{eV})$')
ax2.set_ylabel('$\\frac{1}{N}\\frac{dN}{d\\lg{\\left(E/\\mathrm{eV}\\right)}}$')
ax2.set_ylim([1e-5, 2])
ax2.set_xlim([4.5, 10.5])
plt.tight_layout()

# plt.savefig("Espec_EPOS_p_lgE19_theta_r_dep.pdf", bbox_inches='tight')

In [ ]:
def get_mean_std_showers(df, key):
    df_gb = df.groupby(key)
    df_mean = df_gb.mean().reset_index()
    df_std = df_gb.std().reset_index()
    n = df_gb.count()['lgE'].iloc[0]
    return df_mean, df_std, n

Plots of parameters vs lgr and theta(proton)

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15, 6))

df_p_Z0 = df_p.query('theta == 0')

df_p_mean_Z0_lgr, df_p_std_Z0_lgr, n_p_Z0_lgr = get_mean_std_showers(df_p_Z0, 'lgr')

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, file in enumerate(df_p_Z0['filename'].unique()):
    
    df_ = df_p_Z0.loc[df_p_Z0['filename'] == file]

    axes[0].hlines(df_['ry'], df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)
    axes[1].hlines(df_['Ee']/1e6, df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)
    axes[2].hlines(df_['Ey']/1e6, df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)

rylabel = '$r^\\gamma$'
Eelabel = '$\\langle E^e \\rangle$ [MeV]'
Eylabel = '$\\langle E^\\gamma \\rangle$ [MeV]'
axes[0].set_ylabel(rylabel)
axes[1].set_ylabel(Eelabel)
axes[2].set_ylabel(Eylabel)

df_mean, df_std, n = df_p_mean_Z0_lgr, df_p_std_Z0_lgr, n_p_Z0_lgr

axes[0].errorbar(df_mean['lgr'], df_mean['ry'],
                 xerr=0.02, yerr=df_std['ry']/np.sqrt(n),
                 ls='', color='k', lw=3)
axes[1].errorbar(df_mean['lgr'], df_mean['Ee']/1e6, xerr=0.02,
                 yerr=df_std['Ee']/np.sqrt(n)/1e6,
                 ls='', color='k', lw=3)
axes[2].errorbar(df_mean['lgr'], df_mean['Ey']/1e6,
                 xerr=0.02, yerr=df_std['Ey']/np.sqrt(n)/1e6,
                 ls='', color='k', lw=3)
for ax in axes:
    ax.set_xlabel('$\\log_{10}(r/\mathrm{m})$')
plt.tight_layout()

# plt.savefig('ry_Ee_Ey_vs_r.pdf', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11, 6))


colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, file in enumerate(df_p_Z0['filename'].unique()):
    df_ = df_p_Z0.loc[df_p_Z0['filename'] == file]
    axes[0].hlines(df_['Emu1sq'], df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)
    axes[1].hlines(df_['fmu2'], df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)


axes[0].errorbar(df_mean['lgr'], df_mean['Emu1sq'], xerr=0.02,
                 yerr=df_std['Emu1sq']/np.sqrt(n),
                 ls='', color='k', lw=3)
axes[1].errorbar(df_mean['lgr'], df_mean['fmu2'], xerr=0.02, yerr=df_std['fmu2']/np.sqrt(n),
                 ls='', color='k', lw=3)

Emu12label = '$\\langle  \\left(E^\\mu_1\\right)^2 \\rangle / E_{\\rm VEM}^2$'
fmu2label = '$f^\mu_2$'
axes[0].set_ylabel(Emu12label)
axes[1].set_ylabel(fmu2label)
for ax in axes:
    ax.set_xlabel('$\\log_{10}(r/\mathrm{m})$')

plt.tight_layout()
# plt.savefig('Emu21_fmu2_vs_r.pdf', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15, 6))

df_p_r1000 = df_p.query('lgrh == 3')

df_mean_r1000_theta, df_std_r1000_theta, n_r1000_theta = get_mean_std_showers(df_p_r1000, 'theta')


colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, Xmax in enumerate(df_p_r1000['Xmax'].unique()):
    df_ = df_p_r1000.loc[df_p_r1000['Xmax'] == Xmax]
    axes[0].plot(df_['theta'], df_['ry'], marker='o', ls='', ms=4)
    axes[1].plot(df_['theta'], df_['Ee']/1e6, marker='o', ls='', ms=4)
    axes[2].plot(df_['theta'], df_['Ey']/1e6, marker='o', ls='', ms=4)
    
axes[0].set_ylabel(rylabel)
axes[1].set_ylabel(Eelabel)
axes[2].set_ylabel(Eylabel)

df_mean, df_std, n = df_mean_r1000_theta, df_std_r1000_theta, n_r1000_theta

axes[0].errorbar(df_mean['theta'], df_mean['ry'], yerr=df_std['ry']/np.sqrt(n),
                 ls='', color='k', zorder=99, marker='o', ms=10, lw=3)
axes[1].errorbar(df_mean['theta'], df_mean['Ee']/1e6,yerr=df_std['Ee']/np.sqrt(n)/1e6,
                  ls='', color='k', zorder=99, marker='o', ms=10, lw=3)
axes[2].errorbar(df_mean['theta'], df_mean['Ey']/1e6, yerr=df_std['Ey']/np.sqrt(n)/1e6,
                 ls='', color='k', zorder=99, marker='o', ms=10, lw=3)
for ax in axes:
    ax.set_xlabel('$\\theta$ [deg]')
plt.tight_layout()
# plt.savefig('ry_Ee_Ey_vs_theta.pdf', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11, 6))

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, Xmax in enumerate(df_p_r1000['Xmax'].unique()):
    df_ = df_p_r1000.loc[df_p_r1000['Xmax'] == Xmax]
    axes[0].plot(df_['theta'], df_['Emu1sq'], marker='o', ls='', ms=4)
    axes[1].plot(df_['theta'], df_['fmu2'], marker='o', ls='', ms=4)


axes[0].errorbar(df_mean['theta'], df_mean['Emu1sq'], yerr=df_std['Emu1sq']/np.sqrt(n),
                 ls='', color='k', zorder=99, marker='o', ms=10, lw=4)
axes[1].errorbar(df_mean['theta'], df_mean['fmu2'], yerr=df_std['fmu2']/np.sqrt(n),
                  ls='', color='k', zorder=99, marker='o', ms=10, lw=4)
axes[0].set_ylabel(Emu12label)
axes[1].set_ylabel(fmu2label)
for ax in axes:
    ax.set_xlabel('$\\theta$ [deg]')
plt.tight_layout()
# plt.savefig('Emu21_fmu2_vs_theta.pdf', bbox_inches='tight')

In [ ]:
## RUN this to get alpha/beta
eps_mu_ssd = 1
eps_y_ssd = 0.037
eps_e_ssd = 1

eps_mu_wcd_low = 1
eps_mu_wcd_high = 1
eps_y_wcd = 0.89
eps_e_wcd = 1

G = 0.3772561614030111

ssdfactor = 1.13
ssd_mu_factor = 1

#Swcd = a*Nmu + Ne*(b + c*ry)
#Sssd = Aw/As * (Nmu + Ne*(1+f*ry))

Lwcd = 1
for df in [df_p, df_Fe]:
    df['alpha'] = ssdfactor * G * EVEM * (eps_e_ssd + eps_y_ssd*df['ry'])/(df['Ee']*eps_e_wcd + df['ry']*df['Ey']*eps_y_wcd)
    df['beta'] = ssdfactor * G * eps_mu_ssd / (df['Emu1sq']*eps_mu_wcd_low + df['fmu2']*eps_mu_wcd_high)
    df['a'] = df['Emu1sq'] + df['fmu2']
    df['b'] = df['Ee']/EVEM
    df['c'] = df['Ey']/EVEM
    df['f'] = eps_y_ssd
    df['ssdfactor'] = 1.13 * G
    df['ssd_mu_factor'] = 1.25
    

In [ ]:
df['beta'].mean(), df['alpha'].mean()

In [ ]:
def wcd_ssd_model(Nmu, Ne, a, b, c, f, ry,
                  Aw=10.2, As=3.84, ssd_factor=1.13, ssd_mu_factor=1.25):
    Swcd = a*Nmu + Ne*(b+c*ry)
    Sssd = As/Aw * ssd_factor * (ssd_mu_factor*Nmu + Ne*(1+f*ry))
    return Swcd, Sssd

In [ ]:
Nmu = 10
Ne = 0

wcd_ssd_model(Nmu, Ne, df_p['a'].mean(), df_p['b'].mean(), df_p['c'].mean(), df_p['f'].mean(), 
             df_p['ry'].mean())

In [ ]:
from time_templates.utilities.fitting import fit_poly

f, axes = plt.subplots(1,4, figsize=(15, 5))

dict_fit = defaultdict(list)
for theta in sorted(df_p['theta'].unique()):
    df_ = df_p.query(f'theta == {theta}')
    df_gb = df_.groupby('lgr')
    df_mean = df_gb.mean().reset_index()
    rerr = (df_mean['lgrh'] - df_mean['lgrl'])/2
    n = df_gb.count().reset_index()
    df_err = df_gb.std().reset_index()/np.sqrt(n)
    
    for ax, key in zip(axes, ['a', 'b', 'c', 'ry']):
        x = df_mean['lgr'].values
        y = df_mean[key].values
        yerr = df_err[key].values
        pl = ax.errorbar(x, y, yerr=yerr, ls='', marker='s',
                    label=f'$\\theta = {np.rad2deg(theta):.0f}^\\circ$')
        ax.set_ylabel(key)
        
#         if key == 'a' or key == 'b':
#             deg = 1
#         else:
#             deg = 2

        mask = (x > np.log10(500)) & (x < np.log10(2000))
        
        fitp, fitperr, chi2, ndof = fit_poly(x[mask]-3, y[mask], yerr[mask], deg=2)
        fitp = fitp[::-1]
        xspace = np.linspace(x[0], x[-1])
        ax.plot(xspace, np.poly1d(fitp[::-1])(xspace-3), color=pl[0].get_color())
        #reversed order
        
        for i, p in enumerate(fitp):
            dict_fit[key+f'p{i}'].append(p)
            dict_fit[key+f'perr{i}'].append(fitperr[i])
    axes[0].legend(fontsize=14)
    
plt.tight_layout()

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))

df_p_Z0 = df_p.query('theta == 0')

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for i, file in enumerate(df_p_Z0['filename'].unique()):
    df_ = df_p_Z0.loc[df_p_Z0['filename'] == file]

    ax1.hlines(df_['alpha'], df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)
    ax2.hlines(df_['beta'], df_['lgrl'], df_['lgrh'], color=colors[i], lw=1)

df_p_mean_Z0_lgr, df_p_std_Z0_lgr, n_p_Z0_lgr = get_mean_std_showers(df_p_Z0, 'lgr')

df_mean, df_std, n = df_p_mean_Z0_lgr, df_p_std_Z0_lgr, n_p_Z0_lgr

ax1.errorbar(df_mean['lgr'], df_mean['alpha'], xerr=0.02, yerr=df_std['alpha']/np.sqrt(n),
                 ls='', color='k', lw=3)
ax2.errorbar(df_mean['lgr'], df_mean['beta'], xerr=0.02, yerr=df_std['beta']/np.sqrt(n),
                 ls='', color='k', lw=3)


ax1.set_xlabel('$\\log_{10}(r/\mathrm{m})$')
ax2.set_xlabel('$\\log_{10}(r/\mathrm{m})$')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}$')
plt.tight_layout()
# plt.savefig('alpha_beta_vs_r_toy.pdf', bbox_inches='tight')

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

df_p_r1000 = df_p.query('lgrl == 3')

for i, Xmax in enumerate(df_p_r1000['Xmax'].unique()):
    df_ = df_p_r1000.loc[df_p_r1000['Xmax'] == Xmax]
    ax1.plot(df_['theta'], df_['alpha'], marker='o', ls='', ms=4)
    ax2.plot(df_['theta'], df_['beta'], marker='o', ls='', ms=4)

df_p_mean_r1000_theta, df_p_std_r1000_theta, n_p_r1000_theta = get_mean_std_showers(df_p_r1000, 'theta')

df_mean, df_std, n = df_p_mean_r1000_theta, df_p_std_r1000_theta, n_r1000_theta 


ax1.errorbar(df_mean['theta'], df_mean['alpha'], yerr=df_std['alpha']/np.sqrt(n),
                 ls='', color='k', zorder=99, marker='o', ms=10, lw=4)
ax2.errorbar(df_mean['theta'], df_mean['beta'], yerr=df_std['beta']/np.sqrt(n),
                  ls='', color='k', zorder=99, marker='o', ms=10, lw=4)

plt.tight_layout()
ax1.set_xlabel('$\\theta$ [deg]')
ax2.set_xlabel('$\\theta$ [deg]')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}$')
plt.tight_layout()
# plt.savefig('alpha_beta_vs_theta_toy.pdf', bbox_inches='tight')

Compare proton and iron

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))

df_Fe_Z0 = df_Fe.query('theta == 0')

df_Fe_mean_Z0_lgr, df_Fe_std_Z0_lgr, n_Fe_Z0_lgr = get_mean_std_showers(df_Fe_Z0, 'lgr')

ax1.errorbar(df_p_mean_Z0_lgr['lgr'], df_p_mean_Z0_lgr['alpha'],
             xerr=0.02, yerr=df_p_std_Z0_lgr['alpha']/np.sqrt(n_p_Z0_lgr),
                 ls='', color='b', lw=3, label='proton')

ax1.errorbar(df_Fe_mean_Z0_lgr['lgr'], df_Fe_mean_Z0_lgr['alpha'],
             xerr=0.02, yerr=df_Fe_std_Z0_lgr['alpha']/np.sqrt(n_Fe_Z0_lgr),
                 ls='', color='r', lw=3, label='iron')

ax2.errorbar(df_p_mean_Z0_lgr['lgr'], df_p_mean_Z0_lgr['beta'],
             xerr=0.02, yerr=df_p_std_Z0_lgr['beta']/np.sqrt(n_p_Z0_lgr),
                 ls='', color='b', lw=3)

ax2.errorbar(df_Fe_mean_Z0_lgr['lgr'], df_Fe_mean_Z0_lgr['beta'],
             xerr=0.02, yerr=df_Fe_std_Z0_lgr['beta']/np.sqrt(n_Fe_Z0_lgr),
                 ls='', color='r', lw=3)

ax1.legend(fontsize=16)
ax1.set_xlabel('$\\log_{10}(r/\mathrm{m})$')
ax2.set_xlabel('$\\log_{10}(r/\mathrm{m})$')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}$')
plt.tight_layout()
# plt.savefig('alpha_beta_vs_r_toy_proton_iron.pdf', bbox_inches='tight')

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))

ax1.errorbar(df_p_mean_r1000_theta['theta'], df_p_mean_r1000_theta['alpha'],
             yerr=df_p_std_r1000_theta['alpha']/np.sqrt(n_p_r1000_theta),
                 ls='', color='b', zorder=99, marker='o', ms=10, lw=4)
ax2.errorbar(df_p_mean_r1000_theta['theta'], df_p_mean_r1000_theta['beta'],
             yerr=df_p_std_r1000_theta['beta']/np.sqrt(n_p_r1000_theta),
                  ls='', color='b', zorder=99, marker='o', ms=10, lw=4)

df_Fe_r1000 = df_Fe.query('lgrh == 3')

df_Fe_mean_r1000_theta, df_Fe_std_r1000_theta, n_Fe_r1000_theta = get_mean_std_showers(df_Fe_r1000, 'theta')

ax1.errorbar(df_Fe_mean_r1000_theta['theta'], df_Fe_mean_r1000_theta['alpha'],
             yerr=df_Fe_std_r1000_theta['alpha']/np.sqrt(n_Fe_r1000_theta),
                 ls='', color='r', zorder=99, marker='o', ms=10, lw=4)
ax2.errorbar(df_Fe_mean_r1000_theta['theta'], df_Fe_mean_r1000_theta['beta'],
             yerr=df_Fe_std_r1000_theta['beta']/np.sqrt(n_Fe_r1000_theta),
                  ls='', color='r', zorder=99, marker='o', ms=10, lw=4)


plt.tight_layout()
ax1.set_xlabel('$\\theta$ [deg]')
ax2.set_xlabel('$\\theta$ [deg]')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}$')
plt.tight_layout()
# plt.savefig('alpha_beta_vs_theta_toy_proton_iron.pdf', bbox_inches='tight')

compare EPOS and QGSII

In [ ]:
#QGSII
df_p_Q = read_files(data_dir+'/QGSII/proton')
df_Fe_Q = read_files(data_dir+'/QGSII/iron')
for df in [df_p_Q, df_Fe_Q]:
    df['alpha'] = ssdfactor * G * EVEM * (eps_e_ssd + eps_y_ssd*df['ry'])/(df['Ee'] + df['ry']*df['Ey'])
    df['beta'] = ssdfactor * G * ssd_mu_factor / (df['Emu1sq'] + df['fmu2'])

In [ ]:
from uncertainties import ufloat, unumpy

names = ['EPOS p', 'EPOS Fe', 'QGSJET p', 'QGSJET Fe']
colors = {name: c for name, c in zip(names, ['b', 'r', 'b', 'r'])}
mfcs = {name: mfc for name, mfc in zip(names, ['b', 'r', 'none', 'none'])}
rys = []
Ees = []
Eys = []
Emu12s = []
fmu2s = []

f, axes = plt.subplots(1, 5, figsize=(16, 3.5))

theta = np.deg2rad(0)

for i, df in enumerate([df_p, df_Fe, df_p_Q, df_Fe_Q]):
    name = names[i]
    means = df.query(f'theta == {theta} & lgrh ==3').mean()
    std = df.query(f'theta == {theta} & lgrh ==3').std()
    n = df.query(f'theta == {theta} & lgrh ==3').shape[0]
    ry = means['ry']
    Ee = means['Ee']/1e6
    Ey = means['Ey']/1e6
    Emu12 = means['Emu1sq']
    fmu2 = means['fmu2']
    ry_err = std['ry']/np.sqrt(n)
    Ee_err = std['Ee']/np.sqrt(n)/1e6
    Ey_err = std['Ey']/np.sqrt(n)/1e6
    Emu12_err = std['Emu1sq']/np.sqrt(n)
    fmu2_err = std['fmu2']/np.sqrt(n)
    ry = ufloat(ry, ry_err)
    Ee = ufloat(Ee, Ee_err)
    Ey = ufloat(Ey, Ey_err)
    Emu12 = ufloat(Emu12, Emu12_err)
    fmu2 = ufloat(fmu2, fmu2_err)
    rys.append(ry)
    Ees.append(Ee)
    Eys.append(Ey)
    Emu12s.append(Emu12)
    fmu2s.append(fmu2)

    print('---------------')
    print(name)
    print(f'$r^\\gamma = {ry:.2f}$, $E^e = {Ee:.2f}$ MeV,  $E^\\gamma = {Ey:.2f}$ MeV')
    print(f'(Emu1/Evem)^2 = {Emu12:.3f}, $f^\mu_2 = {fmu2:.3f}$') 

    c = colors[name]
    mfc = mfcs[name]
    axes[0].errorbar(i, ry.nominal_value, yerr=ry.std_dev, marker='s', color=c, mfc=mfc,
               capsize=2)
    axes[1].errorbar(i, Ee.nominal_value, yerr=Ee.std_dev, marker='s', color=c, mfc=mfc,
               capsize=2)
    axes[2].errorbar(i, Ey.nominal_value, yerr=Ey.std_dev, marker='s', color=c, mfc=mfc, capsize=2)
    axes[3].errorbar(i, Emu12.nominal_value, yerr=Emu12.std_dev, marker='s', color=c, mfc=mfc,
               capsize=2)
    axes[4].errorbar(i, fmu2.nominal_value, yerr=fmu2.std_dev, marker='s', color=c, mfc=mfc,
               capsize=2)
    
for ax in axes:
    ax.get_xaxis().set_ticks([])
    ax.set_xlim([-5, 7])


axes[0].set_ylabel(rylabel)
axes[1].set_ylabel(Eelabel)
axes[2].set_ylabel(Eylabel)
axes[3].set_ylabel(Emu12label)
axes[4].set_ylabel(fmu2label)

plt.tight_layout()
# plt.savefig('parameters_toy_r1000_Z0_compare_him_primary.pdf', bbox_inches='tight')

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))


ax1.errorbar(df_p_mean_Z0_lgr['lgr'], df_p_mean_Z0_lgr['alpha'],
             xerr=0., yerr=df_p_std_Z0_lgr['alpha']/np.sqrt(n_p_Z0_lgr),
                 ls='', color='b', lw=2, label='EPOS-LHC proton', capsize=4, marker='s')

# ax1.errorbar(df_Fe_mean_Z0_lgr['lgr'], df_Fe_mean_Z0_lgr['alpha'],
#              xerr=0.02, yerr=df_Fe_std_Z0_lgr['alpha']/np.sqrt(n_Fe_Z0_lgr),
#                  ls='', color='r', lw=3, label='iron')

ax2.errorbar(df_p_mean_Z0_lgr['lgr'], df_p_mean_Z0_lgr['beta'],
             xerr=0, yerr=df_p_std_Z0_lgr['beta']/np.sqrt(n_p_Z0_lgr),
                 ls='', color='b', lw=2, capsize=4, marker='s')

# ax2.errorbar(df_Fe_mean_Z0_lgr['lgr'], df_Fe_mean_Z0_lgr['beta'],
#              xerr=0.02, yerr=df_Fe_std_Z0_lgr['beta']/np.sqrt(n_Fe_Z0_lgr),
#                  ls='', color='r', lw=3)

df_p_Q_Z0 = df_p_Q.query('theta == 0')

df_p_Q_mean_Z0_lgr, df_p_Q_std_Z0_lgr, n_p_Q_Z0_lgr = get_mean_std_showers(df_p_Q_Z0, 'lgr')

ax1.errorbar(df_p_Q_mean_Z0_lgr['lgr'], df_p_Q_mean_Z0_lgr['alpha'],
             xerr=0., yerr=df_p_Q_std_Z0_lgr['alpha']/np.sqrt(n_p_Q_Z0_lgr),
                 ls='', color='b', lw=2, label='QGSJET-II.04 proton', marker='s', capsize=4,
            mfc='none')

ax2.errorbar(df_p_Q_mean_Z0_lgr['lgr'], df_p_Q_mean_Z0_lgr['beta'],
             xerr=0., yerr=df_p_Q_std_Z0_lgr['beta']/np.sqrt(n_p_Q_Z0_lgr),
                 ls='', color='b', lw=2, marker='s', capsize=4, mfc='none')

ax1.legend(fontsize=16)
ax1.set_xlabel('$\\log_{10}(r/\mathrm{m})$')
ax2.set_xlabel('$\\log_{10}(r/\mathrm{m})$')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}$')
plt.tight_layout()
# plt.savefig('alpha_beta_vs_r_toy_EPOS_QGS.pdf', bbox_inches='tight')

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))


ax1.errorbar(df_p_mean_r1000_theta['theta'], df_p_mean_r1000_theta['alpha'],
             yerr=df_p_std_r1000_theta['alpha']/np.sqrt(n_p_r1000_theta),
                 ls='', color='b', zorder=99, marker='o', ms=10, lw=2, capsize=4, label='EPOS-LHC proton')
ax2.errorbar(df_p_mean_r1000_theta['theta'], df_p_mean_r1000_theta['beta'],
             yerr=df_p_std_r1000_theta['beta']/np.sqrt(n_p_r1000_theta),
                  ls='', color='b', zorder=99, marker='o', ms=10, lw=2, capsize=4)
            

df_p_Q_r1000 = df_p_Q.query('lgrh == 3')

df_p_Q_mean_r1000_theta, df_p_Q_std_r1000_theta, n_p_Q_r1000_theta = get_mean_std_showers(df_p_Q_r1000, 'theta')

ax1.errorbar(df_p_Q_mean_r1000_theta['theta'], df_p_Q_mean_r1000_theta['alpha'],
             yerr=df_p_Q_std_r1000_theta['alpha']/np.sqrt(n_p_Q_r1000_theta),
                 ls='', color='g', zorder=99, marker='o', ms=10, lw=2, capsize=4, label='QGSJET-II.04 proton')
ax2.errorbar(df_p_Q_mean_r1000_theta['theta'], df_p_Q_mean_r1000_theta['beta'],
             yerr=df_p_Q_std_r1000_theta['beta']/np.sqrt(n_p_Q_r1000_theta),
                  ls='', color='g', zorder=99, marker='o', ms=10, lw=2, capsize=4)

ax1.legend(fontsize=14)
ax1.set_xlabel('$\\theta$ [deg]')
ax2.set_xlabel('$\\theta$ [deg]')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}(r=1000\, \\rm m)$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}(r=1000\, \\rm m)$')
plt.tight_layout()
# plt.savefig('alpha_beta_vs_theta_toy_EPOS_QGS.pdf', bbox_inches='tight')

Compare signal vs toy model

In [ ]:
from time_templates.datareader.get_data import fetch_MC_data_from_tree
import numpy as np
def load_rec_data(fl, **kwargs):
    df_rec = fetch_MC_data_from_tree(fl, **kwargs)
    df_rec['MCr'] = df_rec['MCr'].apply(round)
    df_rec['MCCosTheta'] = df_rec['MCCosTheta'].round(2)
    df_rec['MCTheta'] = df_rec['MCTheta'].apply(lambda x: np.round(x, 4))
#     df_rec = df_rec.query('MClgE == 19.0')
    print(df_rec.shape)
    return df_rec


In [ ]:


df_rec_p_Q = load_rec_data('/home/mart/auger/data/CorSims_aab/UUB_recs/tree/tree_UUB_SSD_Aab_QGSII_proton_36_rings.root', univ_comp=False, no_traces=True, dense=True)
df_rec_Fe_Q = load_rec_data('/home/mart/auger/data/CorSims_aab/UUB_recs/tree/tree_UUB_SSD_Aab_QGSII_iron_36_rings.root', univ_comp=False, no_traces=True, dense=True)
df_rec_p_E = load_rec_data('/home/mart/auger/data/CorSims_aab/UUB_recs/tree/tree_UUB_SSD_Aab_EPOS_proton_36_rings.root', univ_comp=False, no_traces=True, dense=True)
df_rec_Fe_E = load_rec_data('/home/mart/auger/data/CorSims_aab/UUB_recs/tree/tree_UUB_SSD_Aab_EPOS_iron_36_rings.root', univ_comp=False, no_traces=True, dense=True)

In [ ]:
def get_uncertainty_pandas(x, n):
    s1 = np.sum(x)
    s2 = np.sum(x**2)
    
    std = np.sqrt(s2/n - (s1/n)**2)
    return std/np.sqrt(n)

def calc_uncertainty_ab(wcd, ssd, wcd_err, ssd_err):
    ab = ssd/wcd
    corr = 0.9
    return ab * np.sqrt(wcd_err**2/wcd**2 + ssd_err**2/ssd**2 - corr*wcd_err/wcd * ssd_err/ssd)


def make_alpha_beta_plot(df, cosTheta, lgE, axes=None, **pltkwargs):
    df_q = df.query(f'MCCosTheta > {cosTheta-0.01} & MCCosTheta < {cosTheta+0.02} & MClgE > {lgE-0.1} & MClgE < {lgE+0.1}')
    df_gb = df_q.groupby(['MCr'])
    #beware: do not get mean, but sum and divide by number of stations before trigger: (12 * nsims=10)
    df_gb_mean = df_gb.sum().reset_index()
    r = df_gb_mean['MCr'].values
    m = df_gb.count()['MClgE'].iloc[0]
    
    ssd_em_signal = df_gb_mean['SSDEMSignal'].values/m
    wcd_em_signal = df_gb_mean['WCDEMSignal'].values/m
    ssd_muon_signal = df_gb_mean['SSDMuonSignal'].values/m
    wcd_muon_signal = df_gb_mean['WCDMuonSignal'].values/m

    ssd_em_signal_err = df_gb['SSDEMSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    wcd_em_signal_err = df_gb['WCDEMSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    ssd_muon_signal_err = df_gb['SSDMuonSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    wcd_muon_signal_err = df_gb['WCDMuonSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values

    alpha = ssd_em_signal/wcd_em_signal
    beta = ssd_muon_signal/wcd_muon_signal

    alpha_err = calc_uncertainty_ab(wcd_em_signal, ssd_em_signal,
                                    wcd_em_signal_err, ssd_em_signal_err)
    beta_err = calc_uncertainty_ab(wcd_muon_signal, ssd_muon_signal,
                                   wcd_muon_signal_err, ssd_muon_signal_err)

    if axes is None:
        f, axes = plt.subplots(1, 2, figsize=(15, 7))

    axes[0].errorbar(r, alpha, alpha_err, **pltkwargs)

    axes[1].errorbar(r, beta, beta_err, **pltkwargs)

    axes[0].set_xlabel('r [m]')
    axes[1].set_xlabel('r [m]')
    axes[0].set_ylabel('alpha')
    axes[1].set_ylabel('beta')
    return r, alpha, alpha_err, beta, beta_err

def make_alpha_beta_plot_vs_theta(df, r, lgE, axes=None, **pltkwargs):
    df_q = df.query(f'MClgE > {lgE-0.1} & MClgE < {lgE+0.1} & MCr == {r}')
    df_gb = df_q.groupby(['MCCosTheta'])
    #beware: do not get mean, but sum and divide by number of stations before trigger: (12 * nsims=10)
    df_gb_mean = df_gb.sum().reset_index()
    theta = np.arccos(df_gb_mean['MCCosTheta'].values)
    m = df_gb.count()['MClgE'].iloc[0]
    
    ssd_em_signal = df_gb_mean['SSDEMSignal'].values/m
    wcd_em_signal = df_gb_mean['WCDEMSignal'].values/m
    ssd_muon_signal = df_gb_mean['SSDMuonSignal'].values/m
    wcd_muon_signal = df_gb_mean['WCDMuonSignal'].values/m

    ssd_em_signal_err = df_gb['SSDEMSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    wcd_em_signal_err = df_gb['WCDEMSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    ssd_muon_signal_err = df_gb['SSDMuonSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values
    wcd_muon_signal_err = df_gb['WCDMuonSignal'].apply(lambda x: get_uncertainty_pandas(x, m)).values

    alpha = ssd_em_signal/wcd_em_signal
    beta = ssd_muon_signal/wcd_muon_signal

    alpha_err = calc_uncertainty_ab(wcd_em_signal, ssd_em_signal, wcd_em_signal_err, ssd_em_signal_err)
    beta_err = calc_uncertainty_ab(wcd_muon_signal, ssd_muon_signal, wcd_muon_signal_err, ssd_muon_signal_err)

    if axes is None:
        f, axes = plt.subplots(1, 2, figsize=(15, 7))

    axes[0].errorbar(np.rad2deg(theta), alpha, alpha_err, **pltkwargs)

    axes[1].errorbar(np.rad2deg(theta), beta, beta_err, **pltkwargs)

    axes[0].set_xlabel('$\\theta$ [deg]')
    axes[1].set_xlabel('$\\theta$ [deg]')
    axes[0].set_ylabel('alpha')
    axes[1].set_ylabel('beta')
    return theta, alpha, alpha_err, beta, beta_err

EPOS LHC proton

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11.2, 6), sharex=True)

ax1, ax2 = axes

make_alpha_beta_plot_vs_theta(df_rec_p_E, 1000, 19, axes=axes, ls='', marker='s');

ax1.errorbar(np.rad2deg(df_p_mean_r1000_theta['theta']), df_p_mean_r1000_theta['alpha'],
             yerr=df_p_std_r1000_theta['alpha']/np.sqrt(n_p_r1000_theta),
                 ls='', color='k', zorder=99, marker='o',
             label='from toy model ($1000 < r/\mathrm{m} < 1095$)')
ax2.errorbar(np.rad2deg(df_p_mean_r1000_theta['theta']), df_p_mean_r1000_theta['beta'],
             yerr=df_p_std_r1000_theta['beta']/np.sqrt(n_p_r1000_theta),
                  ls='', color='k', zorder=99, marker='o')

ax1.errorbar([0], [0], marker='s', color='C0', ls='', label='from reconstructed signal ($r=1000\, \\rm m$)')
ax1.legend(fontsize=14)
ax1.set_xlabel('$\\theta$ [deg]')
ax2.set_xlabel('$\\theta$ [deg]')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}(r=1000\, \\rm m) $')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}(r=1000\, \\rm m)$')

ax1.set_ylim([1.1, 1.7])
ax2.set_ylim([0.45, 0.85])
ax1.set_xlim([-5, 50])
plt.tight_layout()
# plt.savefig('alpha_beta_vs_theta_toy_signal_1000m.pdf', bbox_inches='tight')

Discrepancy between rec and toy is due to the following:
SSD signal is underestimated
for EM: low energy electrons can be stopped and make more signal than 1 MIP, protons are in this case taken into EM part. 
for Muon: low energy muons can be stopped and make more signal than 1 MIP
Interesting: the difference ratio seems to be the same for alpha and beta.
Also on average the area and track length cancel, but the track-length mean != mode, so could be that something strange is happening


In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11, 6), sharex=True)

ax1, ax2 = axes

xerr = 10**df_p_mean_Z0_lgr['lgr'] *(10**0.02 - 1)
ax1.errorbar(10**df_p_mean_Z0_lgr['lgr'], df_p_mean_Z0_lgr['alpha'],
             xerr=xerr, yerr=df_p_std_Z0_lgr['alpha']/np.sqrt(n_p_Z0_lgr),
                 ls='', color='k', lw=2)
ax2.errorbar(10**df_p_mean_Z0_lgr['lgr'], df_p_mean_Z0_lgr['beta'],
             xerr=xerr, yerr=df_p_std_Z0_lgr['alpha']/np.sqrt(n_p_Z0_lgr),
                 ls='', color='k', lw=2)

rb, _, _, beta, beta_err = make_alpha_beta_plot(df_rec_p_E, np.cos(np.deg2rad(0)), 19, axes=axes, marker='s', ls='');

ax1.set_xlabel('$r\ [\\rm m]$')
ax2.set_xlabel('$r\ [\\rm m]$')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}$')
plt.tight_layout()


ax1.errorbar([0], [0], marker='', xerr=0.5, yerr=0.5, label='from toy model', color='k')
ax1.errorbar([0], [0], marker='s', yerr=0.5, xerr=0, label='from reconstructed signal',
             color='C0')
ax1.legend(fontsize=16)
ax1.set_xlim([350, 1600])
ax1.set_ylim([1.0, 1.75])
ax2.set_ylim([0.45, 0.8])
# plt.savefig('alpha_beta_vs_r_toy_and_signal.pdf', bbox_inches='tight')


In [ ]:
np.arange(0, 360, 360/10)

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 7))

ax.errorbar(10**df_p_mean_Z0_lgr['lgr'], df_p_mean_Z0_lgr['beta'],
             xerr=xerr, yerr=df_p_std_Z0_lgr['alpha']/np.sqrt(n_p_Z0_lgr),
                 ls='', color='k', lw=2, label='from MC muon energy spectrum')

ax.errorbar(rb, beta, yerr=beta_err, marker='s', ls='', color='C0', label='from signal ratio')
ax.set_ylabel('$\\beta = \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}} \\approx 0.38 \\frac{N^\\mu_{\\mathrm{WCD}}} {S^\\mu_{\\mathrm{WCD}}}$',
             fontsize=24)
ax.set_xlabel('$r\ [\\rm m]$')

from time_templates.templates.muon_dsdt import I1_interp, I2_interp, I3_interp

rspace = np.linspace(400, 2500)
Evem_GeV = Evem/1e9 + 0.105

for l in [2000, 4000, 8000]:
    out = np.zeros_like(rspace)
    for i, r in enumerate(rspace):
        out[i] = 0.378 * I3_interp(np.log10(l), np.log10(r), Evem_GeV) / \
            (I1_interp(np.log10(l), np.log10(r), Evem_GeV) + I2_interp(np.log10(l), np.log10(r), Evem_GeV)) 
        
    ax.plot(rspace, out, ls='--', label=f'$l = {l:.0f}$ m')
    
ax.legend(frameon=True, ncol=2)
ax.set_ylim([0.3, 1.2])
# plt.savefig('beta_muon_model_energy.pdf', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11, 6), sharex=True)
ax1, ax2 = axes
make_alpha_beta_plot(df_rec_p_E, np.cos(np.deg2rad(0)), 19, axes=axes, marker='s', ls='', color='b',
                    label='EPOS proton');
# make_alpha_beta_plot(df_rec_Fe_E, np.cos(np.deg2rad(0)), 19, axes=axes, marker='s', ls='', color='r',
#                     label='EPOS iron');

make_alpha_beta_plot(df_rec_p_Q, np.cos(np.deg2rad(0)), 19, axes=axes, marker='s', ls='', color='b',
                     mfc='none', label='QGSJET proton');
# make_alpha_beta_plot(df_rec_Fe_Q, np.cos(np.deg2rad(0)), 19, axes=axes, marker='s', ls='', color='r',
#                      mfc='none', label='QGSJET iron');

ax1.set_xlim([350, 2000])
ax1.set_ylim([1.2, 1.75])
ax2.set_ylim([0.45, 0.8])

ax1.set_xlabel('$r\ [\\rm m]$')
ax2.set_xlabel('$r\ [\\rm m]$')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}$')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}$')
plt.tight_layout()


plt.savefig('compare_EPOS_QGS_alpha_beta.pdf', bbox_inches='tight')


In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11, 6), sharex=True)

ax1, ax2 = axes

make_alpha_beta_plot_vs_theta(df_rec_p_E, 1000, 19, axes=axes, ls='', marker='s', color='b', 
                             label='EPOS proton');
make_alpha_beta_plot_vs_theta(df_rec_Fe_E, 1000, 19, axes=axes, ls='', marker='s', color='r',
                             label='EPOS iron');
make_alpha_beta_plot_vs_theta(df_rec_p_Q, 1000, 19, axes=axes, ls='', marker='s', color='b', mfc='none',
                             label='QGSJET proton');
make_alpha_beta_plot_vs_theta(df_rec_Fe_Q, 1000, 19, axes=axes, ls='', marker='s', color='r', mfc='none',
                             label='QGSJET iron');

ax1.legend(fontsize=16)
ax1.set_xlabel('$\\theta$ [deg]')
ax2.set_xlabel('$\\theta$ [deg]')

ax1.set_ylabel('$\\alpha = \\frac{S^{\\rm em}_{\\rm SSD}}{S^{\\rm em}_{\\rm WCD}}(r=1000\, \\rm m) $')
ax2.set_ylabel('$\\beta= \\frac{S^{\\mu}_{\\rm SSD}}{S^{\\mu}_{\\rm WCD}}(r=1000\, \\rm m)$')

ax1.set_ylim([1.3, 1.85])
ax2.set_ylim([0.5, 0.85])
ax1.set_xlim([-5, 50])
plt.tight_layout()

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

dict_ab = {}

for i, df in enumerate([df_rec_p_E, df_rec_Fe_E, df_rec_p_Q, df_rec_Fe_Q]):
    df = df.query('MCr == 1000 & MCCosTheta >0.99')
    name = names[i]
    df_mean = df.mean()
    df_std = df.std()
    n = df.count()['MCr']
    alpha = df_mean['SSDEMSignal']/df_mean['WCDEMSignal']
    beta = df_mean['SSDMuonSignal']/df_mean['WCDMuonSignal']
    print(df_mean['WCDEMSignal'], df_mean['WCDMuonSignal'])
    alpha_err = calc_uncertainty_ab(df_mean['WCDEMSignal'], df_mean['SSDEMSignal'],
                                   df_std['WCDEMSignal']/np.sqrt(n),
                                   df_std['SSDEMSignal']/np.sqrt(n))
    beta_err = calc_uncertainty_ab(df_mean['WCDMuonSignal'], df_mean['SSDMuonSignal'],
                               df_std['WCDMuonSignal']/np.sqrt(n),
                               df_std['SSDMuonSignal']/np.sqrt(n))
    print('----------------')
    print(names[i])
    print('alpha: ', ufloat(alpha, alpha_err))
    print('beta: ', ufloat(beta, beta_err))
    c = colors[name]
    mfc = mfcs[name]
    dict_ab[names[i]] = {}
    dict_ab[names[i]]['alpha'] = ufloat(alpha, alpha_err)
    dict_ab[names[i]]['beta'] = ufloat(beta, beta_err)
    ax1.errorbar(i, alpha, alpha_err, marker='s', color=c, mfc=mfc, capsize=4)
    ax2.errorbar(i, beta, beta_err, marker='s', color=c, mfc=mfc, capsize=4, label=name)
    if not 'Fe' in name:
        ax2.annotate(name, xy=(i, beta),
                     textcoords='offset points', xytext=(8, -5), fontsize=16)

ax2.annotate('EPOS Fe', xy=(-4, 0.66),  fontsize=16)
ax2.annotate('QGSJET Fe', xy=(-3, 0.635), fontsize=16)

ax1.get_xaxis().set_ticks([])
ax2.get_xaxis().set_ticks([])
ax1.set_xlim([-5, 8])
ax2.set_xlim([-5, 8])
# ax2.set_ylim([0.625, 0.8])
ax1.set_ylabel('$\\alpha$')
ax2.set_ylabel('$\\beta$')
plt.tight_layout()
plt.savefig('alpha_beta_signal_r1000_Z0_compare_HIM_primary.pdf', bbox_inches='tight')


\begin{align}
\begin{pmatrix}
 \Swcd \\
 \Sssd
 \end{pmatrix} = 
 \begin{pmatrix}
1 & 1 \\
\alpha & \beta 
 \end{pmatrix}
 \begin{pmatrix}
\Swcdem \\
 \Swcdmu
 \end{pmatrix}
\end{align}

\begin{align}
\begin{pmatrix}
\Swcdem \\
\Swcdmu
\end{pmatrix} = 
\frac{1}{\beta - \alpha}
\begin{pmatrix}
\beta & -1 \\
-\alpha & 1
\end{pmatrix}
\begin{pmatrix}
\Swcd \\
\Sssd
\end{pmatrix}
\end{align}

In [ ]:
def matrix_inversion(Swcd, Sssd, alpha, beta):
    Swcdem = 1/(beta-alpha) * (beta*Swcd - Sssd)
    Swcdmu = 1/(beta-alpha) * (Sssd - alpha*Swcd)
    return Swcdem, Swcdmu


#at r=1000 and theta=0
Swcdmu_true = 17 #from EPOS proton
Swcdem_true = 27
alpha_true = 1.501
beta_true = 0.721
Swcd = Swcdmu_true + Swcdem_true
Sssd = alpha_true*Swcdem_true + beta_true*Swcdmu_true
print(Swcd, Sssd)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))


for i, key in enumerate(dict_ab):
    alpha = dict_ab[key]['alpha']
    beta = dict_ab[key]['beta']
    Swcdem, Swcdmu = matrix_inversion(Swcd, Sssd, alpha, beta)
    c = colors[key]
    mfc = mfcs[key]
    ax1.errorbar(i, Swcdem.nominal_value, yerr=Swcdem.std_dev, marker='s', color=c, mfc=mfc, capsize=2)
    ax2.errorbar(i, Swcdmu.nominal_value, yerr=Swcdmu.std_dev, marker='s', color=c, mfc=mfc, capsize=2)

ax1.get_xaxis().set_ticks([])
ax2.get_xaxis().set_ticks([])
ax1.set_xlim([-5, 8])
ax2.set_xlim([-5, 8])

ax1.set_ylabel('$\\hat{S}_{\\rm WCD}^{\\rm em}$ [VEM]')
ax2.set_ylabel('$\\hat{S}_{\\rm WCD}^{\\mu}$ [VEM]')
ax1.axhline(Swcdem_true, ls='--', color='k')
ax2.axhline(Swcdmu_true, ls='--', color='k')
ax1.annotate('true', xy=(-4.5, Swcdem_true+0.08) )
ax2.annotate('true', xy=(-4.5, Swcdmu_true+0.08), )

plt.tight_layout()

In [ ]:
Swcdmu_true = 17
Swcdem_true = 27
alpha_true = 1.501
beta_true = 0.721
Swcd = Swcdmu_true + Swcdem_true
Sssd = alpha_true*Swcdem_true + beta_true*Swcdmu_true

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))


for i, key in enumerate(dict_ab):
    alpha = dict_ab[key]['alpha']
    beta = dict_ab[key]['beta']
    Swcdem, Swcdmu = matrix_inversion(Swcd, Sssd, alpha, beta)
    c = colors[key]
    mfc = mfcs[key]
    ax1.errorbar(i, Swcdem.nominal_value/Swcdem_true, yerr=Swcdem.std_dev/Swcdem_true,
                 marker='s', color=c, mfc=mfc, capsize=2)
    ax2.errorbar(i, Swcdmu.nominal_value/Swcdmu_true, yerr=Swcdmu.std_dev/Swcdmu_true,
                 marker='s', color=c, mfc=mfc, capsize=2)

ax1.get_xaxis().set_ticks([])
ax2.get_xaxis().set_ticks([])
ax1.set_xlim([-5, 8])
ax2.set_xlim([-5, 8])

ax1.set_ylabel('$\\hat{S}_{\\rm WCD}^{\\rm em} / S_{\\rm WCD}^{\\rm em}$ ')
ax2.set_ylabel('$\\hat{S}_{\\rm WCD}^{\\mu} / S_{\\rm WCD}^{\\mu} $')
ax1.axhline(1, ls='--', color='k')
ax2.axhline(1, ls='--', color='k')


plt.tight_layout()

In [ ]:
def matrix_inv(Swcd, Sssd,
               Ee=27, Eph=6.8, ry=25,
               Emu12=0.118, fmu2=0.66,
               eps_y_ssd=0.03, Evem=400, Awcd=10.2, Assd=3.84):
    """
    Swcd = a*Nmu + Ne*(b + ry*c)
    Sssd = Assd/Awcd * (d*Nmu + Ne*(e + ry*f))
    """
    
    a = Emu12 + fmu2
    b = Ee/Evem
    c = Eph/Evem
    d = 1
    e = 1
    f = eps_y_ssd
    G = Awcd/Assd
    det = (a-b-c*ry + a*f*ry)
    Nmu = ( Swcd*(1+f*ry) -  (G * Sssd * (b+c*ry)) )/det
    Ne = (G * Sssd * a - Swcd ) / det
    iterable = True
    try:
        iter(Nmu)
    except:
        iterable = False
        
    if not iterable:
        if Nmu < 0:
            Nmu = np.nan
        if Ne < 0:
            Ne = np.nan
    else:
        Nmu[Nmu < 0] = np.nan
        Ne[Ne < 0] = np.nan
    return Nmu, Ne

In [ ]:
Emu12 = 0.1
fmu2 = 0.7
Ee = 27
Eph = 7
Evem = 400
eps = 0
ry = 25

Awcd = 10.2
Assd = 3.84

a = Emu12 + fmu2
b = Ee/Evem
c = Eph/Evem
d = 1
e = 1
f = eps

# CHECK

Nmu = 10
Ne = 10

Sw = a*Nmu + Ne*(b+ry*c)
Ss = Assd/Awcd * (d*Nmu + Ne*(e+ ry*f))

print(Sw, Ss)
print(matrix_inv(Sw, Ss, Ee, Eph, ry, Emu12, fmu2, eps, Evem, Awcd, Assd))

In [ ]:
def variance_signal(Swcd, Sssd, Ee=27, Eph=6.8, ry=25,
                    Emu12=0.118, fmu2=0.66,
                    eps_y_ssd=0.03, Evem=400, Awcd=10.2, Assd=3.84):
    """
    Swcd = a*Nmu + Ne*(b + ry*c)
    Sssd = d*Nmu + Ne*(e + ry*f)
    """
    
    a = Emu12 + fmu2
    b = Ee/Evem
    c = Eph/Evem
    d = 1
    e = 1
    f = eps_y_ssd
    Nmu, Ne = matrix_inv(Swcd, Sssd, Ee, Eph, ry,
                         Emu12, fmu2,
                         eps_y_ssd, Evem,
                         Awcd, Assd)
    sigma_w2 = Nmu*a**2 + Ne*(b**2 + ry*c**2)
    sigma_s2 = (Assd/Awcd) * (Nmu + Ne*(1+ry*f**2))
    return sigma_w2, sigma_s2
    
def variance_signal_2(Swcd, Sssd, Ee=27, Eph=6.8, ry=25,
                    Emu12=0.118, fmu2=0.66,
                    eps_y_ssd=0.03, Evem=400, Awcd=10.2, Assd=3.84):
    
    a = Emu12 + fmu2
    b = Ee/Evem
    c = Eph/Evem
    d = 1
    e = 1
    f = eps_y_ssd
    
    det = a-b-c*ry + a*f*ry
    #these are checked to be correct with inputting matrix inversion into variance
    Cw1 = (a**2*(1+f*ry) - c**2*ry - b**2) / det
    Cw2 = a*(b**2 + c**2*ry - a*(b+c*ry)) * Awcd/Assd / det
    

    Cs1 = f*ry*(1-f) / det * Assd/Awcd
    Cs2 = (a-b-c*ry+a*f**2*ry) / det
    
    print(Cw1, Cw2, Cs1, Cs2)
    
    return Cw1 * Swcd + Cw2 * Sssd, Cs1*Swcd + Cs2*Sssd

In [ ]:
variance_signal_2(1, 1)

In [ ]:
Swcd = np.logspace(1, 2.3, 100)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 5))


for fssd in [0.5, 0.8, 1, 1.2]:
    sigma_w, sigma_s = np.sqrt(variance_signal(Swcd, Swcd*fssd))
    sigma_w2, sigma_s2 = np.sqrt(variance_signal_2(Swcd, Swcd*fssd))
    pl = ax1.plot(Swcd, sigma_w, label=fssd)
    ax2.plot(fssd*Swcd, sigma_s, color=pl[0].get_color())
    ax1.plot(Swcd, sigma_w2, label=fssd, ls='--', color=pl[0].get_color())
    ax2.plot(fssd*Swcd, sigma_s2, ls='--', color=pl[0].get_color())   
ax1.legend()
ax1.set_xlabel('Swcd')
# ax1.set_xscale('log')
ax2.set_xlabel('Sssd')
# ax2.set_xscale('log')

In [ ]:
from matplotlib.colors import LogNorm
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))

Swcd = np.logspace(1, 3, 100)
Sssd = np.linspace(0.4, 1.4, 100)
Swg, Ssg = np.meshgrid(Swcd, Swcd)

Ssg = Swcd * Sssd[:, None]

sig_w, sig_s = np.sqrt(variance_signal(Swg, Ssg))

im = ax1.imshow(sig_w/Swg, origin='lower', extent=[Swcd[0], Swcd[-1], Sssd[0], Sssd[-1]],
                aspect='auto', vmin=0, vmax=0.3)
cb = f.colorbar(im, ax=ax1)
cb.set_label('$\\sigma_{\\rm WCD}/S $')
im = ax2.imshow(sig_s/Ssg, origin='lower',  extent=[Swcd[0], Swcd[-1], Sssd[0], Sssd[-1]],
                aspect='auto', vmin=0, vmax=0.3)
cb = f.colorbar(im, ax=ax2)
cb.set_label('$\\sigma_{\\rm SSD}/S $')

ax1.set_xlabel('wcd signal')
ax2.set_xlabel('wcd signal')
ax1.set_ylabel('ssd signal frac')
ax2.set_ylabel('ssd signal frac')
ax1.set_xscale('log')
# ax1.set_yscale('log')

ax2.set_xscale('log')
# ax2.set_yscale('log')
# 
plt.tight_layout()

In [ ]:
from scipy.integrate import quad

def muon_dNdE(Ekin, r, psi, theta, zmax, pa=0.0002, gamma=2.7, kappa=0.8, Q=0.17):
    E = Ekin + 0.105 #GeV
    Delta = r*np.tan(theta)*np.cos(psi)
    lmax = np.sqrt(r**2 + (zmax-Delta)**2)
    pal = pa*lmax
    return (E+pal)**(2-gamma) * (E/(E+pal))**kappa * np.exp(-E*r/(lmax*Q))


Ekin = np.logspace(-2, 3)

f, ax = plt.subplots(1)

for zmax in [3000, 6000, 15000]:

    ax.plot(Ekin, muon_dNdE(Ekin, 1000, 0, 0, zmax))
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim([1e-4, 1])

In [ ]:
def E12_VEM(r, psi, theta, zmax, Evem=0.4):
    func = lambda x: muon_dNdE(x, r, psi, theta, zmax)
    N = quad(func, 0, np.inf)[0]
    E12 = quad(lambda x: (x/Evem)**2*func(x)/N, 0, Evem)[0]/N
    fmu2 = quad(func, Evem, np.inf)[0]/N
    return E12, fmu2
    
E12_VEM(1000, 0, 0, 10000)

In [ ]:
n = 20
r = np.logspace(2.5, 3.3, n)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))

for zmax in [4000, 8000, 16000]:
    out = np.zeros((n, 2))

    for i, _r in enumerate(r):
        out[i, :] = E12_VEM(_r, 0, 0, zmax)



    ax1.plot(r, out[:, 0])
    ax2.plot(r, out[:, 1])